জীবনানন্দ দাশের মতোন কবিতা লেখার চেষ্টা করা যাক!

জীবনানন্দের সমস্ত কবিতা [এইখান](https://www.bangla-kobita.com/jibanananda/) থেকে স্ক্র্যাপ করে র ডেটাসেট তৈরি করছি। 

### Pre-processing

ড্রাইভ থেকে ডেটাসেট ডাউনলোড করলাম।

In [1]:
!wget -q https://gdurl.com/s26P/download
!mv download jida.txt

এইবার প্রিপ্রসেসিং। টেক্স ডেটা নিয়া কাজ করবার সবচাইতে গুরুত্বপূর্ণ কাজ ঝাড়ু দেয়া।

In [2]:
with open('jida.txt') as f:
  raw = f.read()
  
raw = [r for r in raw.split('\n') if 'কাব্যগ্রন্থ' not in r]
raw = ' '.join(raw)

In [3]:
raw = raw.translate(str.maketrans('', '', '–!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~“”\u200c—।১২৩৪৫৬৭৮৯০\xa0'))
raw = raw.replace("\'", "’")
tokens = raw.split()

এমনভাবে লাইন-শব্দগুলোকে সাজানো হলো যাতে বিশটা শব্দ ট্রেইনিংয়ে গিয়ে শেষ শব্দ প্রেডিক্ট করা যায়। ডেটাসেটের কয়েক লাইন দেখলে স্পষ্ট হয়ে যাবে।

In [4]:
seq_length = 20 + 1
sequences = []
for i in range(seq_length, len(tokens)):
  seq = tokens[i-seq_length:i]
  line = ' '.join(seq)
  sequences.append(line)

ঝাড়ু দিয়ে আবর্জনা দূর করে নতুন ডেটাসেট তৈরি করলাম।

In [5]:
with open('jida_processed.txt', 'w') as f:
  doc = '\n'.join(sequences)
  f.write(doc)

### Language Model

প্রসেস করা ডেটা পেয়ে গেলে এবার গৎবাধা কাজ। মডেল তৈরি করে সেখানে ডেটা ফিট করা। আর্কিটেকচারে যেসব লেয়ার নেয়া হলো- তার বিস্তারিত লেখাপড়া দরকার। নইলে ক্যানো-কিভাবে কাজ করতেছে- বোঝা যাবে না।

In [6]:
from numpy import array
from pickle import dump

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [7]:
with open('jida_processed.txt') as f:
  doc = f.read()
lines = doc.split('\n')

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [9]:
len(sequences), len(sequences[0])

(71038, 21)

In [10]:
vocab_size = len(tokenizer.word_index) + 1

In [11]:
sequences = array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.21)

In [13]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape, 

((56120, 20), (14918, 20), (56120, 14393), (14918, 14393))

In [14]:
# modified
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(Bidirectional(LSTM(170, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(150))
model.add(Dense(300, activation='relu'))
model.add(Dense(600, activation='relu'))
model.add(Dense(1200, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 50)            719650    
_________________________________________________________________
bidirectional (Bidirectional (None, 20, 340)           300560    
_________________________________________________________________
dropout (Dropout)            (None, 20, 340)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 150)               294600    
_________________________________________________________________
dense (Dense)                (None, 300)               45300     
_________________________________________________________________
dense_1 (Dense)              (None, 600)               180600    
_________________________________________________________________
dense_2 (Dense)              (None, 1200)              7

In [15]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

earlyStopping = EarlyStopping(monitor='loss', patience=21, verbose=0, mode='min')
model_save = ModelCheckpoint('model_light.h5', save_best_only=True, monitor='loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')

In [16]:
history = model.fit(X, y, batch_size=200, epochs=500, verbose=1,
                    callbacks=[earlyStopping, model_save, reduce_lr_loss])

Epoch 1/500
356/356 [==============================] - 14s 39ms/step - loss: 8.3107 - accuracy: 0.0080 - lr: 0.0010
Epoch 2/500
356/356 [==============================] - 14s 39ms/step - loss: 7.9721 - accuracy: 0.0102 - lr: 0.0010
Epoch 3/500
356/356 [==============================] - 14s 39ms/step - loss: 7.8090 - accuracy: 0.0107 - lr: 0.0010
Epoch 4/500
356/356 [==============================] - 14s 39ms/step - loss: 7.6539 - accuracy: 0.0124 - lr: 0.0010
Epoch 5/500
356/356 [==============================] - 14s 40ms/step - loss: 7.4679 - accuracy: 0.0155 - lr: 0.0010
Epoch 6/500
356/356 [==============================] - 14s 40ms/step - loss: 7.2386 - accuracy: 0.0205 - lr: 0.0010
Epoch 7/500
356/356 [==============================] - 14s 40ms/step - loss: 6.9793 - accuracy: 0.0270 - lr: 0.0010
Epoch 8/500
356/356 [==============================] - 14s 40ms/step - loss: 6.7020 - accuracy: 0.0345 - lr: 0.0010
Epoch 9/500
356/356 [==============================] - 14s 40ms/step - l

### Generating Prose

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [18]:
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [19]:
!cp model_light.h5 '/content/drive/My Drive/Colab Notebooks/'
!cp tokenizer.pkl '/content/drive/My Drive/Colab Notebooks/'

In [20]:
from random import randint
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
with open('jida_processed.txt') as f:
  doc = f.read()
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

In [ ]:
# to download model_ligth.h5 and tokenizer.pkl (if you don't want to train)
# !wget -qq https://drive.google.com/u/0/uc?id=1MqmaI_vQysOOQ1IyKuxWGNXpvC8adAYZ&export=download
# !mv "/content/uc?id=1MqmaI_vQysOOQ1IyKuxWGNXpvC8adAYZ" model_light.h5
# !wget -qq https://drive.google.com/u/0/uc?id=1OeqZF59Oop0IJO-MJB0SOvjwACdHNbZp&export=download
# !mv "/content/uc?id=1OeqZF59Oop0IJO-MJB0SOvjwACdHNbZp" tokenizer.pkl

In [22]:
from tensorflow.keras.models import load_model
from pickle import load

model = load_model('model_light.h5')
tokenizer = load(open('tokenizer.pkl', 'rb'))

In [23]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

প্রথম কয়েকটা শব্দ ধরায় দিলে দিব্বি জীদার ভাষায় কবিতা বের হতে থাকবে এখন।

In [24]:
seed_text = lines[randint(0, len(lines))]
seed_text

'কাজ কিবা হায় বিয়ে হয়েছিল কবেমরে গেছে বউ যদিও মহুয়া গাছে ফুটে ওঠে মৌ একবার ঝরে গেলে তবু তারপর মহুয়া মহুয়া'

প্রতিবার পাঁচশো শব্দ উৎপাদন হচ্ছে- দশ শব্দে প্রতি লাইন পদ্যের ভাব আনতে। নইলে গদ্যও পড়া চলে।

In [29]:
seed_text = 'গ্রীষ্মের সমুদ্র থেকে চোখের ঘুমের গান আসিতেছে ভেসে'
generated = generate_seq(model, tokenizer, seq_length, seed_text, 500)

i = 0
for w in generated.split():
  print(w, end=' ')
  i += 1
  if not i % 10:
    print()

এখানে পালঙ্কে শুয়ে কাটিবে অনেক দিন জেগে থেকে ঘুমাবার সাধ 
ভালোবেসে তবুও যখন মৃত্যু হবে উপস্থিত আরএকটি প্রভাতের হয়তো বা 
অন্যতর বিস্তীর্ণতায় মনে হবে অনেক প্রতীক্ষা মোরা করে গেছি পৃথিবীতে 
চোয়ালের মাংস ক্রমে ক্ষীণ করে কোনো এক বিশীর্ণ কাকের অক্ষিগোলকের 
সাথে আঁখিতারকার সব সমাহার এক দেখে তবু লঘু হাস্যে সন্তানের 
জন্ম দিয়ে তারা আমাদের মতো হবে সেই কথা জেনে ভুলে 
গিয়ে লোল হাস্যে জলের তরঙ্গ মোরা শুনে গেছি আমাদের প্রাণের 
ভিতর নব শিকড়ের স্বাদ অনুভব করে গেছি ভোরের স্ফটিক রৌদ্রে 
অনেক গন্ধর্ব নাগ কুকুর কিন্নর পঙ্গপাল বহুবিধ জন্তুর কপাল উন্মোচিত 
হয়ে বিরুদ্ধে দাঁড়ায়ে থাকে পথ পথান্তরে তবু ওই নীলিমাকে প্রিয় 
অভিভাবিকার মতো মনে হয় হাতে তার তুলাদণ্ড শান্ত স্থির মুখের 
প্রতিজ্ঞাপাশে নির্জন নীলাভ বৃত্তি ছাড়া কিছু নেই যেন তার কাছে 
জীবনের অভ্যুদয় মধ্য সমুদ্রের পরে অনুকূল বাতাসের প্ররোচনাময় কোনো এক 
ক্রীড়া ক্রীড়া বেরিলমণির মতো তরঙ্গের উজ্জ্বল আঘাতে মৃত্যু স্থির শুভ্র 
নৈসর্গিক কথা বলিবার অবসর অশ্বত্থ বটের পথে অনেক হয়েছি আমি 
তোমাদের সাথী ছড়ায়েছি খই ধান বহুদিন উঠানের শালিখের তরে সন্ধ্যায় 
পুকুর থেকে হাঁসটি

In [28]:
seed_text = 'মরিবার হলো তার সাধ'
generated = generate_seq(model, tokenizer, seq_length, seed_text, 500)

i = 0
for w in generated.split():
  print(w, end=' ')
  i += 1
  if not i % 10:
    print()

মাসবেহুলার লহনার মধুর জগতে আজ একদিন ঘরে দুই চোখ ঠোট 
নাসিকা আঙুল তাহার ছোয়াচে ভিজে গেছে চুল শাদা শাদা ফেনাফুলে 
কত বার দূর উপকূলে তারাভরা আকাশের তলে বালকের মতো এক 
সমুদ্রের জলে দেহ ধুয়ে নিয়া জেনেছি দেহের স্বাদ গেছে বুক 
মুখ পরশিয়া রাঙা রোদ নারীর মতন এ দেহ পেয়েছে যেন 
তাহার চুম্বন ফসলের ক্ষেতে প্রথম প্রণয়ী সে যে কার্তিকের ভোরবেলা 
দূরে যেতে যেতে থেমে গেছে সে আমার তরে চোখ দুটো 
ফের ঘুমে ভরে যেন তার চুমো খেয়ে এ দেহ অলস 
মেয়ে পুরুষের সোহাগে অবশ চুমে লয় রৌদ্রের রস হেমন্ত বৈকালে 
উড়ো পাখাপাখালির পালে উঠানের পেতে থাকে কান শোনো ঝরা শিশিরের 
গান অঘ্রানের মাঝরাতে হিম হাওয়া যেন শাদা কঙ্কালের হাতে এ 
দেহেরে এসে ধরে ব্যথা দেয় নারীর অধরে চুলে চোখে জুঁয়ের 
নিশ্বাসে ঝুমকো লতার মতো তার দেহ ফাঁসে ভরা ফসলের মতো 
পড়ে ছিঁড়ে এই দেহ ব্যথা পায় ফিরে… তবু এই শস্যক্ষেতে 
পিপাসার ভাষা ফুরাবে না কে বা সেই চাষা কাস্তে হাতে 
কঠিন কামুক আমাদের সবটুকু ব্যথাভরা সুখ উচ্ছেদ করিবে এসে একা 
কে বা সেই জানি না তো হয় নাই দেখা আজও 
তার সনে আজ শুধু দেহ আর দেহের পীড়নে সাধ মোর 
চোখে ঠোঁটে চুলে শুধু পীড়া শুধু পীড়া মুকুলে মুকুলে শু